In [1]:
!pip install -U tensorflow_datasets

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=ca2a4be7b2301d5f9bf341853ee8506b419d7c627f5791cbdf90de35a5b77b6e
  Stored in directory: c:\users\aesir\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [3]:
pip install tensorflow

  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import math
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_ds, test_ds = dataset['train'], dataset['test']

Dataset fashion_mnist downloaded and prepared to C:\Users\AEsir\tensorflow_datasets\fashion_mnist\3.0.1. Subsequent calls will reuse this data.


In [7]:
metadata

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='C:\\Users\\AEsir\\tensorflow_datasets\\fashion_mnist\\3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-077

In [8]:
labels = metadata.features['label'].names
num_test_examples = metadata.splits['test'].num_examples
num_train_examples = metadata.splits['train'].num_examples

In [10]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)

train_ds = train_ds.cache()
test_ds = test_ds.cache()

In [12]:
basic_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [16]:
basic_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [17]:
BATCH_SIZE = 32
train_batch = train_ds.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_batch = test_ds.cache().batch(BATCH_SIZE)

In [21]:
# set training epochs set to 1
basic_model.fit(train_batch, epochs=1, steps_per_epoch=math.ceil(num_train_examples / BATCH_SIZE))

1875/1875 [==============================] - 6s 2ms/step - loss: 0.4941 - accuracy: 0.8270


In [24]:
test_loss, test_accuracy = basic_model.evaluate(test_batch, steps = math.ceil(num_test_examples / BATCH_SIZE))
print(test_accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.4352 - accuracy: 0.8428
0.8428000211715698


In [ ]:
# accuracy will be lower than 5 epochs which is about 87% in test accuracy 
#  and 89% in training accuracy 

In [56]:
def diff_neurons(neuron_num):
    print('number of neurons in the dense layer: {}'.format(neuron_num))
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(neuron_num, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])
    model.fit(train_batch, epochs=5, steps_per_epoch=math.ceil(num_train_examples / BATCH_SIZE))
    test_losses, test_accuracy = model.evaluate(test_batch, steps = math.ceil(num_test_examples / BATCH_SIZE))
    return test_accuracy

In [57]:
neuron_array = [10, 100, 256, 512]
accuracy = []
for num in neuron_array:
    accuracy.append(diff_neurons(num))

number of neurons in the dense layer: 10
Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6986 - accuracy: 0.7664
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4715 - accuracy: 0.8376
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4424 - accuracy: 0.8456
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4336 - accuracy: 0.8487
Epoch 5/5
313/313 [==============================] - 0s 920us/step - loss: 0.4726 - accuracy: 0.8358
number of neurons in the dense layer: 100
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4987 - accuracy: 0.8245
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3703 - accuracy: 0.8669
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3408 - accuracy: 0.8753
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3157 - accuracy: 0.8847
Epoch 5/

In [58]:
data = {'num of neuron': neuron_array, 'accuracy': accuracy}
pd.DataFrame(data=data)

,num of neuron,accuracy
0,10,0.8358
1,100,0.8685
2,256,0.8748
3,512,0.8741


In [35]:
# different neurons in additional layer
def diff_neurons_add_layer(neuron_num):
    print('number of neurons in the dense layer: {}'.format(neuron_num))
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(neuron_num, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])
    model.fit(train_batch, epochs=5, steps_per_epoch=math.ceil(num_train_examples / BATCH_SIZE))
    test_losses, test_accuracy = model.evaluate(test_batch, steps = math.ceil(num_test_examples / BATCH_SIZE))
    return test_accuracy

In [46]:
neuron_array = [10, 100, 256, 512]
accuracy = []
for num in neuron_array:
    accuracy.append(diff_neurons_add_layer(num))

number of neurons in the dense layer: 10
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5550 - accuracy: 0.8041
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3847 - accuracy: 0.8607
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3437 - accuracy: 0.8748
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3164 - accuracy: 0.8841
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.3578 - accuracy: 0.8697
number of neurons in the dense layer: 100
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4864 - accuracy: 0.8267
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3616 - accuracy: 0.8663
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3264 - accuracy: 0.8807
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3008 - accuracy: 0.8874
Epoch 5/5


In [39]:
import pandas as pd

In [47]:
data = {'num of neuron': neuron_array, 'accuracy': accuracy}
pd.DataFrame(data=data)

,num of neuron,accuracy
0,10,0.8697
1,100,0.8773
2,256,0.8727
3,512,0.8788


In [54]:
# don't normalize the pixel values
train_no_norm, test_no_norm = dataset['train'], dataset['test']
train_no_norm = train_no_norm.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_no_norm = test_no_norm.cache().batch(BATCH_SIZE)


no_norm_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])
no_norm_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
         metrics=['accuracy'])
no_norm_model.fit(train_no_norm, epochs=5, steps_per_epoch=math.ceil(num_train_examples / BATCH_SIZE))
test_losses, test_accuracy = no_norm_model.evaluate(test_no_norm, steps = math.ceil(num_test_examples / BATCH_SIZE))
print('The test accuracy is: {}'. format(test_accuracy))

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 3.2903 - accuracy: 0.7208
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6254 - accuracy: 0.7808
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5618 - accuracy: 0.7999
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5230 - accuracy: 0.8151
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.5248 - accuracy: 0.8184
The test accuracy is: 0.8184000253677368


In [55]:
data = {'scenario': ['basic model', 'epochs=1', "don't normalize"],
       'accuracy': ['0.8722', '0.8270', '0.8184']}
pd.DataFrame(data=data)

,scenario,accuracy
0,basic model,0.8722
1,epochs=1,0.8270
2,don't normalize,0.8184
